In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix, classification_report

In [2]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

In [3]:
df[['zipcode', 'date']]

,zipcode,date
0,30002,2012-03-31
1,30002,2012-04-30
2,30002,2012-05-31
3,30002,2012-06-30
4,30002,2012-07-31
...,...,...
884087,34698,2023-08-31
884088,34698,2023-09-30
884089,34698,2023-10-31
884090,34698,2023-11-30


In [4]:
df_sorted = df.sort_values(['zipcode', 'date']).reset_index(drop=True)
df_sorted[['zipcode', 'date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [5]:
type(df_sorted['date'][0]), df_sorted['date'][0]

(str, '2012-03-31')

In [6]:
df_sorted['date'] = pd.to_datetime(df_sorted['date'])
df_sorted['date']

0        2012-03-31
1        2012-04-30
2        2012-05-31
3        2012-06-30
4        2012-07-31
            ...    
884087   2023-08-31
884088   2023-09-30
884089   2023-10-31
884090   2023-11-30
884091   2023-12-31
Name: date, Length: 884092, dtype: datetime64[ns]

In [7]:
df_grouped = df_sorted.groupby('zipcode')

In [8]:
ts_all = pd.DataFrame()

ls = [1,3,6,12]

i = 1

#for i in ls:
ts_all[f'target_profitable_{i}m'] = (df_grouped['price'].shift(-i) > 1.02 * df_sorted['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_all['zipcode_obj'] = df_grouped['zipcode'].shift(0).astype('object') # but remains still an integer...
ts_all['date'] = df_sorted['date']
numeric_column_names = df_sorted.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.


for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)

ts_all = ts_all.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])

/tmp/ipykernel_38614/2526076371.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_38614/2526076371.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_38614/2526076371.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

In [9]:
#type(ts_all)
ts_all = ts_all.dropna()
ts_all

,target_profitable_1m,zipcode_obj,date,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
3,0,1431,2012-06-30,177500.0,124000.0,89000.0,207450.0,229500.0,252250.0,120.984975,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,194580.490265,194446.137500,195148.086474
4,0,1431,2012-07-31,217000.0,177500.0,124000.0,203750.0,207450.0,229500.0,121.340580,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,195081.437213,194580.490265,194446.137500
5,0,1431,2012-08-31,130000.0,217000.0,177500.0,193900.0,203750.0,207450.0,81.948640,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,196403.714599,195081.437213,194580.490265
6,0,1431,2012-09-30,130000.0,130000.0,217000.0,203700.0,193900.0,203750.0,95.205795,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,197743.074185,196403.714599,195081.437213
7,0,1431,2012-10-31,191000.0,130000.0,130000.0,162537.5,203700.0,193900.0,118.367237,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,198843.413326,197743.074185,196403.714599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884087,0,98686,2023-08-31,515000.0,530900.0,523227.5,527450.0,520000.0,474900.0,305.216752,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,561259.870706,558047.224949,555511.734585
884088,0,98686,2023-09-30,502900.0,515000.0,530900.0,550000.0,527450.0,520000.0,305.790501,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,564276.110320,561259.870706,558047.224949
884089,0,98686,2023-10-31,509900.0,502900.0,515000.0,511755.0,550000.0,527450.0,311.643836,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,566663.763169,564276.110320,561259.870706
884090,0,98686,2023-11-30,525000.0,509900.0,502900.0,504450.0,511755.0,550000.0,306.527550,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,568266.780424,566663.763169,564276.110320


In [10]:
type(df_sorted['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
df_sorted[['zipcode','date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [12]:
def split_train_predict(df):
    # Ensure it's sorted by zipcode and date
    df = df.sort_values(['zipcode_obj', 'date'])

    # Group by zipcode
    train_list = []
    predict_list = []

    for zipcode, group in df.groupby('zipcode_obj'):
        train_part = group.iloc[:-3]
        predict_part = group.iloc[-3:]

        train_list.append(train_part)
        predict_list.append(predict_part)

    # Concatenate the results
    df_train = pd.concat(train_list).reset_index(drop=True)
    df_predict = pd.concat(predict_list).reset_index(drop=True)

    return df_train, df_predict

# Apply the split
df_train, df_predict = split_train_predict(ts_all)

# Optional: verify
print("Train set:", df_train.shape, 'Type', type(df_train))
print("Predict set:", df_predict.shape, 'Type', type(df_predict))

Train set: (846736, 108) Type <class 'pandas.core.frame.DataFrame'>
Predict set: (18678, 108) Type <class 'pandas.core.frame.DataFrame'>


In [15]:
#df_train
df_predict

,target_profitable_1m,zipcode_obj,date,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,1431,2023-10-31,465000.0,470000.0,522500.0,462450.0,445000.0,479900.0,223.260644,...,3170.0,3170.0,3170.0,3170.0,1541.0,1541.0,1541.0,427411.135867,423577.185071,418786.508703
1,0,1431,2023-11-30,462500.0,465000.0,470000.0,450000.0,462450.0,445000.0,219.758478,...,3170.0,3170.0,3170.0,3170.0,1541.0,1541.0,1541.0,429997.675464,427411.135867,423577.185071
2,0,1431,2023-12-31,469000.0,462500.0,465000.0,398888.0,450000.0,462450.0,200.601023,...,3170.0,3170.0,3170.0,3170.0,1541.0,1541.0,1541.0,431098.634112,429997.675464,427411.135867
3,0,1432,2023-10-31,465000.0,515000.0,530000.0,507450.0,475000.0,449000.0,298.438935,...,7887.0,7887.0,7887.0,7887.0,3905.0,3905.0,3905.0,497218.028527,491574.108994,485074.816725
4,0,1432,2023-11-30,485000.0,465000.0,515000.0,535000.0,507450.0,475000.0,323.688969,...,7887.0,7887.0,7887.0,7887.0,3905.0,3905.0,3905.0,502059.557636,497218.028527,491574.108994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18673,0,98685,2023-11-30,640000.0,605000.0,617332.5,599475.0,648000.0,643950.0,297.304440,...,29456.0,29456.0,29456.0,29456.0,11629.0,11629.0,11629.0,577247.107610,575506.070264,572484.261456
18674,0,98685,2023-12-31,640000.0,640000.0,605000.0,599000.0,599475.0,648000.0,291.176471,...,29456.0,29456.0,29456.0,29456.0,11629.0,11629.0,11629.0,577994.879622,577247.107610,575506.070264
18675,0,98686,2023-10-31,509900.0,502900.0,515000.0,511755.0,550000.0,527450.0,311.643836,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,566663.763169,564276.110320,561259.870706
18676,0,98686,2023-11-30,525000.0,509900.0,502900.0,504450.0,511755.0,550000.0,306.527550,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,568266.780424,566663.763169,564276.110320


In [14]:
grouped_train = df_train.groupby(['zipcode'])
grouped_predict = df_predict.groupby(['zipcode'])

KeyError: 'zipcode'

In [ ]:
ts_train = pd.DataFrame()

ls = [1,3,6,12]

i = 1

#for i in ls:
ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * df_train['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_train['zipcode_obj'] = grouped_train['zipcode'].shift(0).astype('object') # but remains still an integer...
numeric_column_names = df_train.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.


for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

ts_train = ts_train.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])

/tmp/ipykernel_34681/1700554156.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_34681/1700554156.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_34681/1700554156.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [ ]:
ts_train

,target_profitable_1m,zipcode_obj,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,1431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1431,89000.0,NaN,NaN,252250.0,NaN,NaN,65.441176,NaN,...,NaN,2933.0,NaN,NaN,1493.0,NaN,NaN,195148.086474,NaN,NaN
2,0,1431,124000.0,89000.0,NaN,229500.0,252250.0,NaN,120.984975,65.441176,...,NaN,2933.0,2933.0,NaN,1493.0,1493.0,NaN,194446.137500,195148.086474,NaN
3,0,1431,177500.0,124000.0,89000.0,207450.0,229500.0,252250.0,120.984975,120.984975,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,194580.490265,194446.137500,195148.086474
4,0,1431,217000.0,177500.0,124000.0,203750.0,207450.0,229500.0,121.340580,120.984975,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,195081.437213,194580.490265,194446.137500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846731,0,98686,531000.0,555000.0,545000.0,574900.0,510790.0,524900.0,278.898716,281.499086,...,19949.0,19904.0,19949.0,19949.0,7787.0,8205.0,8205.0,554717.026443,557759.329878,559724.449580
846732,0,98686,499900.0,531000.0,555000.0,570445.0,574900.0,510790.0,267.237845,278.898716,...,19949.0,19904.0,19904.0,19949.0,7787.0,7787.0,8205.0,552746.639171,554717.026443,557759.329878
846733,0,98686,499900.0,499900.0,531000.0,520220.0,570445.0,574900.0,268.612521,267.237845,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,552276.246723,552746.639171,554717.026443
846734,0,98686,506000.0,499900.0,499900.0,502560.0,520220.0,570445.0,285.245902,268.612521,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,553637.387867,552276.246723,552746.639171


In [ ]:
ts_test = pd.DataFrame()

#for i in ls:
ts_test[f'target_profitable_{i}m'] = (grouped_predict['price'].shift(-i) > 1.02 * df_predict['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_test['zipcode'] = grouped_predict['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = df_predict.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)

/tmp/ipykernel_34681/3971834618.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)
/tmp/ipykernel_34681/3971834618.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)
/tmp/ipykernel_34681/3971834618.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [ ]:
ts_test

,target_profitable_1m,zipcode,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,1431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1431,522500.0,NaN,NaN,479900.0,NaN,NaN,264.668367,NaN,...,NaN,3170.0,NaN,NaN,1541.0,NaN,NaN,418786.508703,NaN,NaN
2,0,1431,470000.0,522500.0,NaN,445000.0,479900.0,NaN,223.260644,264.668367,...,NaN,3170.0,3170.0,NaN,1541.0,1541.0,NaN,423577.185071,418786.508703,NaN
3,0,1431,465000.0,470000.0,522500.0,462450.0,445000.0,479900.0,223.260644,223.260644,...,3170.0,3170.0,3170.0,3170.0,1541.0,1541.0,1541.0,427411.135867,423577.185071,418786.508703
4,0,1431,462500.0,465000.0,470000.0,450000.0,462450.0,445000.0,219.758478,223.260644,...,3170.0,3170.0,3170.0,3170.0,1541.0,1541.0,1541.0,429997.675464,427411.135867,423577.185071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37351,0,98686,515000.0,NaN,NaN,527450.0,NaN,NaN,305.216752,NaN,...,NaN,19904.0,NaN,NaN,7787.0,NaN,NaN,561259.870706,NaN,NaN
37352,0,98686,502900.0,515000.0,NaN,550000.0,527450.0,NaN,305.790501,305.216752,...,NaN,19904.0,19904.0,NaN,7787.0,7787.0,NaN,564276.110320,561259.870706,NaN
37353,0,98686,509900.0,502900.0,515000.0,511755.0,550000.0,527450.0,311.643836,305.790501,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,566663.763169,564276.110320,561259.870706
37354,0,98686,525000.0,509900.0,502900.0,504450.0,511755.0,550000.0,306.527550,311.643836,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,568266.780424,566663.763169,564276.110320
